---
<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/image_igac.jpg" alt="Imagen_IGAC" width="280">
</p>

---

# ***Fundamentos de LLMs con Python: Explorando ChatGPT y LangChain***

---

#### ***Instructor: [Luis Andrés Campos Maldonado](https://www.linkedin.com/in/lacamposm/)***

##### ***Email: luisandres.campos@igac.gov.co***

##### ***Contratista-Observatorio Inmobiliario Catastral***

---

# ***Clase 09 - 02 de mayo de 2024***
---

## ***Taller Práctico de Implementación con Streamlit - Parte 1***

**Objetivos de Aprendizaje:**

- Revisar brevemente los conceptos clave aprendidos durante el curso y cómo se integrarán en el proyecto final.
- Comenzar el desarrollo de una aplicación con Streamlit que integre las   funcionalidades   de   LangChain,   enfocándose   en   la   estructura básica   y   la   incorporación   de   memoria   y   agentes


## ***Mini proyecto de final de curso***


### ***Descripción de la Tarea.***
Desarrollar una aplicación de chatbot con Streamlit que integre capacidades avanzadas como búsqueda en internet y RAG (Retrieval-Augmented Generation). La aplicación permitirá subir documentos PDF y utilizar la información contenida en estos para generar respuestas contextualizadas, soportadas por un sistema de memoria que permite al chatbot recordar interacciones pasadas.

Esta tarea combina varias tecnologías avanzadas de procesamiento de lenguaje natural, proporcionando experiencia práctica en la creación de aplicaciones interactivas e inteligentes. Al integrar RAG y capacidades de búsqueda, los estudiantes aprenden a manejar y sintetizar grandes volúmenes de información de manera eficaz, mientras que la implementación de memoria en chatbots representa un avance significativo en la creación de sistemas más naturales y útiles para los usuarios finales. Además, desarrollar esta aplicación en Streamlit facilita la visualización y la interacción directa, habilidades esenciales para cualquier desarro en el campo de la AI.

## ***Primera parte.***

Construir una APP que hago de RAG. Use el notebook _APP-RAG-OIC_ [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lacamposm/course-fundamentals-llms-openai-langchain/blob/main/app_final_proyect.ipynb) para la construcción.

## ***Paso 1.***

Cree la primera parte de la interfaz de usuario con el siguiente código:

```python
import os
import shutil
import streamlit as st

os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"
dir_pdf_documents = "RAG"

# Interfaz de usuario #
st.title("🦜 Chatbot with RAG - OIC 🦜")

if "chat_active" not in st.session_state:
    with st.sidebar.expander(f"Seleccione modelo y cargue sus archivos."):
        model_openai = st.radio(
            "Seleccione su ChatModel:",
            ["gpt-3.5-turbo-0125", "gpt-4-0125-preview"],
            key="openai_chat_model"
        )
        st.session_state["openai_model"] = model_openai
        if os.path.exists(dir_pdf_documents):
            delete_dir = st.button(f"Borrar carpeta {dir_pdf_documents}")
            if delete_dir:
                shutil.rmtree(dir_pdf_documents)
                st.rerun()

        uploaded_files = st.file_uploader("Elige los archivos PDF para el RAG", accept_multiple_files=True, type=["pdf"])

        if st.button("Finalizar carga de archivos y Chatear"):
            st.session_state["chat_active"] = True
            if len(uploaded_files) > 0:
                os.makedirs(dir_pdf_documents)
                for uploaded_file in uploaded_files:
                    bytes_data = uploaded_file.read()
                    path_file = os.path.join(dir_pdf_documents, uploaded_file.name)                
                    with open(path_file, "wb") as f:
                        f.write(bytes_data)
                        st.write(f"Archivo {uploaded_file.name} guardado")
                # retriever = create_retriver_documents()
                # st.session_state["retriever"] = retriever
                # st.rerun()
                
            else:
                st.rerun()
# =================================================================================== #
# ====   Creamos sección de chatbot con Memory en session_state de streamlit.  ====== #
# =================================================================================== #
```

Apoye de la lectura: _app-oic - Memory_ [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lacamposm/course-fundamentals-llms-openai-langchain/blob/main/app.ipynb)

Observe las funcionalidades que le ofrece. Comente.

## ***Paso 2:***

Recupere las funciones: 

1. ```chatbot_template```
2. ```stream_response_with_memory_openai```

de: _app-oic - Memory_ [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lacamposm/course-fundamentals-llms-openai-langchain/blob/main/app.ipynb)

Adicionelas justo después de importación de librerias y antes de iniciar la interfaz de usuario.

## ***Paso 3:***

Construya la función ```create_retriver_documents()```:

```python
def create_retriver_documents():
    """
    Crea un recuperador de documentos a partir de archivos PDF almacenados en un directorio especificado.

    Esta función carga documentos desde un directorio utilizando PyPDFDirectoryLoader, divide el
    texto en segmentos manejables mediante RecursiveCharacterTextSplitter y luego incrusta
    estos segmentos usando el modelo de incrustación de texto de OpenAI. Los segmentos de texto incrustados se
    almacenan en un almacenamiento vectorial (Chroma), que luego se utiliza para crear un recuperador para
    la recuperación eficiente de documentos basada en la similitud vectorial.

    Return:
        Retriever: Un objeto capaz de recuperar documentos basado en la similitud de las consultas.
    """
```
Haga uso de las siguientes lineales de código al iniciar la función:

```python
loader = PyPDFDirectoryLoader(dir_pdf_documents)
docs = loader.load()
```

Para el resto de la función tome como guía lo desarrollando en - _Lectura clase 6: RAG_ [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lacamposm/course-fundamentals-llms-openai-langchain/blob/main/Clase_06_retrieval_and_rag.ipynb).

## ***Paso 4.***

Construya la función ```stream_rag_with_memory_openai(model_name, query, chat_history, retriever)```

```python
def stream_rag_with_memory_openai(model_name, query, chat_history, retriever):
    """
    Ejecuta un proceso de generación de respuestas aumentado por la recuperación de información (RAG) 
    con memoria, utilizando el modelo especificado de OpenAI.

    Parámetros:
        model_name (str): El nombre del modelo de OpenAI a utilizar.
        query (str): La consulta del usuario a responder.
        chat_history (list): Historial de chat que puede incluir mensajes previos para mantener el contexto.
        retriever: Un objeto recuperador que accede a la información relevante para responder la consulta.

    Descripción:
    La función configura una cadena de procesamiento donde el contexto y la historia del chat se utilizan para
    formular una respuesta relevante y contextualizada. Se utiliza un prompt específico que considera
    el contexto de catastro en Colombia para estructurar la respuesta.

    Retorna:
        stream: Una cadena de ejecución que procesa la consulta en tiempo real y devuelve una respuesta.
    """
```

Tome como referencia la función ```stream_response_with_memory_openai``` y la _Lectura clase 6: RAG_ [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lacamposm/course-fundamentals-llms-openai-langchain/blob/main/Clase_06_retrieval_and_rag.ipynb), para recuperar el ```rag_chain```. Adicione en el diccionario del pipeline lo siguiente: ```"chat_history": lambda x: chat_history```

Coloque esta función justo después de la también función: ```stream_response_with_memory_openai```

## ***Paso 5.***

Descomente las lineas de codigo en el paso y complete la APP agregando y  :

```python
if "chat_active" in st.session_state and os.path.exists(save_rag_directory):
    st.text(f"Archivos cargados para chatear:\n {', '.join(os.listdir(save_rag_directory))}")

# Iniciamos un chat history #
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

for message in st.session_state.chat_history:
    if isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.markdown(message.content)
    else:
        with st.chat_message("AI"):
            st.markdown(message.content)


user_query = st.chat_input("Your message")

if user_query is not None and user_query != "":
    st.session_state.chat_history.append(HumanMessage(user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):
        if os.path.exists(save_rag_directory):        
            llm_response = st.write_stream(
                stream_rag_with_memory_openai(
                    model_name=st.session_state["openai_model"],
                    query=user_query,
                    chat_history=st.session_state.chat_history,
                    retriever=st.session_state["retriever"]
                    )
            )
        
        else:
            llm_response = st.write_stream(
            stream_response_with_memory_openai(
                model_name=st.session_state["openai_model"],
                query=user_query,
                chat_history=st.session_state.chat_history)
            )
    
    st.session_state.chat_history.append(AIMessage(llm_response))
```

## ***Paso 06***

Haga uso de la APP:

🦜 Chatbot with RAG - OIC 🦜